In [3]:
!pip install customized_konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 881.5/881.5 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 12.8 MB/s eta 0:00:00


In [4]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from ckonlpy.tag import Twitter
import pickle

In [5]:
# 질문리스트 작성(203개)

questions = ['위험물안전관리법의 적용 대상은 무엇인가요?',  #1
            '위험물이란 무엇을 말하나요?',
            '저장소와 취급소의 차이는 무엇인가요?',
            '위험물의 저장 및 취급에 제한이 있는가요?',
            '제조소등에서의 위험물의 저장 및 취급에 어떤 기준을 적용하나요?',
            '위험물안전관리법이 공공의 안전을 위해 목적을 두는 이유는 무엇인가요?',
            '제조소등에서의 위험물의 저장 및 취급에 어떤 기준을 적용하나요?',
            '위험물안전관리법의 목적은 무엇인가요?',
            '제조소등의 위치, 구조, 설비에 관한 기술기준은 누가 정하나요?',
            '위험물안전관리법이 국가의 책무로 정한 사항은 무엇인가요?', #10

            '위험물안전관리법이 적용되지 않는 대상은 어떤 것이 있나요?',
            '위험물의 지정수량이란 무엇을 의미하나요?',
            '위험물의 저장소와 취급소는 어떤 기준에 따라 구분되나요?',
            '위험물의 저장 및 취급에 제한이 있는 경우가 있나요?',
            '위험물의 저장소 및 취급소에서의 안전관리에 어떤 기준을 적용하나요?',
            '위험물안전관리법이 지방자치단체에게 요구하는 사항은 무엇인가요?',
            '제조소등의 위치, 구조, 설비에 대한 기술기준을 정하는 주체는 누구인가요?',
            '위험물안전관리법이 공공의 안전을 위해 두는 목적은 무엇인가요?',
            '위험물의 저장소와 취급소를 구분하는 기준은 무엇인가요?',
            '위험물안전관리법이 국가의 책무로 정한 사항은 어떤 것이 있나요?', #20

            '위험물 저장 또는 취급 시 출입 및 검사를 할 수 있는 권한은 누구에게 있나요?',   #21
            '주행 중인 위험물 운반 차량을 정지시켜 국가기술자격증 또는 교육수료증을 요구할 수 있는 권한은 누구에게 있나요?',    #22
            '소방본부장 또는 소방서장이 출입 및 검사 명령을 내릴 수 있는 대상은 무엇인가요?',   #23
            '위험물 저장소에서 발생한 사고의 원인 및 피해를 조사하는 권한은 누구에게 있나요?',  #24
            '소방공무원이나 경찰공무원이 주행 중인 위험물 운반 차량을 정지시키고 국가기술자격증을 요구하는 상황에서 국가기술자격증을 제시하지 않은 경우, 공무원이 요구하는 것은 무엇인가요?',   #25
            '출입 및 검사 명령을 받은 관계인이 거부한 경우, 공무원은 어떤 조치를 취할 수 있나요?',  #26
            '위험물 저장 또는 취급 시 출입을 제한하는 경우, 개인의 주거에 대해서는 어떤 예외가 적용되나요?',    #27
            '소방본부장 또는 소방서장이 출입 및 검사 명령을 내릴 수 있는 시간은 언제인가요?',   #28
            '출입 및 검사 명령을 수행하는 공무원은 어떤 증표를 지녀야 하나요?', #29
            '위험물 저장소에서 발생한 사고의 조사를 위해 소방청장이 전문지식이 있는 사람으로 구성된 조사위원회를 둘 수 있는 권한은 어디에 있나요?', #30

            '위험물시설의 설치를 위해 필요한 허가를 받는 주체는 누구인가요?',   #31
            '제조소등의 위치, 구조, 설비를 변경하려면 어떤 절차를 거쳐야 하나요?',  #32
            '제조소등의 위치, 구조, 설비 변경 시 변경 전후의 품명, 수량 등을 신고해야 하나요?', #33
            '제조소등에서 저장 또는 취급하는 위험물의 품명 및 수량을 변경하려면 어떤 조치를 취해야 하나요?',    #34
            '어떤 경우에는 제조소등의 설치나 변경에 대한 허가나 신고가 필요하지 않나요?',   #35
            '군부대를 위한 제조소등의 설치나 변경에는 어떤 절차가 적용되나요?', #36
            '위험물탱크 설치나 변경공사를 진행할 때 어떤 절차를 따라야 하나요?',    #37
            '완공검사를 받아야 하는 경우와 그 내용은 어떻게 정해지나요?',   #38
            '제조소등의 설치자가 변경된 경우에는 어떤 절차를 따라야 하나요?',   #39
            '제조소등의 사용을 중지하거나 중지한 후에는 어떤 조치를 해야 하나요?',  #40

            '제조소등 설치나 변경에 따른 완공검사를 받지 않은 경우 어떤 조치를 해야 하나요?',   #41
            '제조소등의 설치자가 변경된 경우 변경 사실을 신고하는 기간은 어떻게 되나요?',   #42
            '위험물안전관리법에 따른 과징금 부과의 기준은 무엇인가요?', #43
            '제조소등의 사용을 중지하거나 재개할 때 어떤 절차를 따라야 하나요?',    #44
            '제조소등의 사용 중지나 재개에 따른 안전조치 확인은 어떻게 이루어지나요?',  #45
            '제조소등의 사용 중지에 따른 위험물안전관리자 선임은 어떤 경우에 예외가 될 수 있나요?', #46
            '제조소등 설치허가 취소나 사용정지에 따른 과징금 부과는 어떤 경우에 이루어지나요?', #47
            '제조소등의 용도 폐지 시 어떤 조치를 취해야 하나요?',   #48
            '제조소등의 사용 중지 또는 재개 시 시ㆍ도지사는 어떤 조치를 취하나요?', #49
            '제조소등 설치허가의 취소나 사용정지에 따라 과징금이 부과되는 경우 어떤 법률에 따라 징수되나요?',   #50

            '위험물시설을 설치하려는 경우에 허가를 받아야 하는 기관은 어디인가요?', #51
            '제조소등에서 저장하거나 취급하는 위험물의 품명ㆍ수량 변경을 신고해야 하는 경우는 언제인가요?', #52
            '군부대가 제조소등을 설치하거나 변경하기 위해 누구와 협의해야 하나요?', #53
            '제조소등에서 위험물탱크의 설치 또는 변경공사를 할 때 어떤 검사를 받아야 하나요?',  #54
            '제조소등의 설치자가 사망하거나 양도/인도한 경우, 그 지위는 누가 승계하나요?',  #55
            '제조소등의 용도를 폐지하기 위해 누구에게 신고해야 하나요?',    #56
            '제조소등의 사용을 중지하려는 경우, 어떤 조치를 해야 하나요?',  #57
            '어떤 경우에 제조소등의 사용 정지나 허가 취소가 이루어질 수 있나요?',#58
            '제조소등의 관계인이 안전조치를 적절히 하지 않을 경우 어떤 조치가 이루어질 수 있나요?', #59
            '안전관리자의 주요 역할은 무엇인가요?', #60

            '안전관리자를 선임하기 위한 조건은 무엇인가요?',    #61
            '안전관리자의 선임 후 해임 및 임명 절차는 어떻게 되나요?',  #62
            '안전관리자의 업무가 중요한 이유는 무엇인가요?',    #63
            '안전관리자가 감독해야 하는 사항은 어떤 것이 있나요?',  #64
            '안전관리자의 업무가 특히 중요한 업종은 무엇인가요?',   #65
            '안전관리자가 특정 사유로 업무를 수행하지 못할 때 대체 수단은 무엇인가요?', #66
            '안전관리자의 업무에 대해 이의를 제기할 수 있는 경우는 언제인가요?',    #67
            '안전관리자가 여행 중이거나 질병 등으로 업무를 수행할 수 없을 때 조치 방안은 무엇인가요?',  #68
            '다수의 제조소에서 하나의 안전관리자를 선임하는 경우 어떤 조치를 취해야 하나요?',#69


##################################################################<윤현우 Part2>###############################################################################################(70~140)
            '안전관리자의 책임은 무엇인가요?',  #70 윤현우 Part
            '안전관리자의 자격 요건은 어떻게 되나요?',  #71
            '안전관리자의 업무 범위는 무엇인가요?',
            '안전관리자의 업무와 관련된 법규는 무엇인가요?',
            '안전관리자가 해야 할 안전점검에는 무엇이 있나요?',
            '안전관리자의 교육 및 경험 요건은 어떻게 되나요?',
            '안전관리자의 역할이 중요한 이유는 무엇인가요?',
            '안전관리자가 해야 할 위험 분석은 어떤 것이 있나요?',
            '안전관리자가 작업자에게 제공해야 할 교육은 무엇인가요?',
            '안전관리자가 작업환경을 안전하게 유지하기 위해 취해야 할 조치는 무엇인가요?',

            '위험물안전관리법은 무엇을 규정하고 있나요?',   #80
            '제15조는 어떤 내용을 다루고 있나요?',
            '안전관리자의 역할은 무엇인가요?',
            '안전관리자를 선임하지 않으면 어떤 문제가 발생할 수 있나요?',
            '안전관리자의 선임에 대한 기준은 무엇인가요?',
            '안전관리자의 선임과 해임에 대해 어떻게 규정되어 있나요?',
            '안전관리자의 직무를 대행하는 경우에 대해 어떻게 규정되어 있나요?',
            '안전관리자의 업무 수행에 관한 규정은 무엇인가요?',
            '안전관리자가 선임되지 않은 경우의 처리 방법은 무엇인가요?',
            '안전관리자의 역할에 대한 존중과 권고에 관한 규정은 무엇인가요?',

            '위험물의 운반은 어떤 기준에 따라 이루어져야 하나요?',  #90
            '위험물을 운반하는 차량의 운전자가 갖추어야 하는 요건은 무엇인가요?',
            '운반용기를 제작하거나 수입한 자의 검사 절차는 어떻게 되나요?',
            '위험물운송자가 지켜야 하는 안전확보 기준은 무엇인가요?',
            '위험물을 운송하는 운송책임자의 역할은 무엇인가요?',
            '이동탱크저장소에 의한 위험물 운송 시 요구되는 조건은 무엇인가요?',
            '위험물의 운반 중 안전사고가 발생했을 때의 대응 절차는 어떻게 되나요?',
            '위험물운송자가 갖추어야 하는 자격은 어떻게 취득하나요?',
            '운송책임자의 감독 또는 지원 방법은 어떻게 되나요?',
            '이동탱크저장소에 의한 위험물 운송 시 적용되는 행정안전부령은 무엇인가요?',

            '위험물 운반 시 중요기준과 세부기준은 무엇인가요?', #100
            '위험물의 운반용기 검사를 받지 않으면 어떤 일이 벌어질까요?',
            '위험물을 운송하는 운송책임자의 주요 역할은 무엇인가요?',
            '위험물의 운반 중 발생할 수 있는 안전사고의 종류는 무엇이 있나요?',
            '위험물 운송책임자의 감독 또는 지원을 받는 방법은 무엇인가요?',
            '운송책임자의 감독 또는 지원을 받는 목적은 무엇인가요?',
            '위험물 운송 중 화재가 발생했을 때의 대응 절차는 어떻게 되나요?',
            '운송용기에 대한 검사를 받는 절차는 어떻게 되나요?',
            '위험물 운송 시 특별한 주의사항은 무엇인가요?',
            '운송책임자가 운송 중 사고 발생 시 책임을 지는 범위는 어떻게 되나요?',

            '위험물 운반 시 중요한 기준은 무엇인가요?', #110
            '위험물의 운송책임자의 주요 역할은 무엇인가요?',
            '위험물의 운반 중 발생할 수 있는 안전사고는?',
            '운송용기에 대한 검사를 받는 절차는?',
            '위험물 운송 중 화재 발생 시 대응 절차는?',
            '위험물을 운반하는 운송책임자의 요건은 무엇인가요?',
            '운송책임자의 감독 또는 지원을 받는 목적은?',
            '운송 중 안전을 보장하기 위한 세심한 주의는?',
            '운송용기의 검사를 받지 않은 경우 어떤 일이 생기나요?',
            '위험물 운송책임자의 감독 또는 지원을 받는 방법은?',

            '위험물의 저장 또는 취급장소에 출입하여 검사하거나 자료를 요구할 수 있는 권한을 가지는 공무원은 누구인가요?',   #120
            '위험물 운반 차량을 정지시켜 운반자의 자격을 확인하는 권한을 가진 공무원은 누구인가요?',
            '출입 및 검사 등의 규정은 보통 어떤 시간에 행해져야 하나요?',
            '출입 및 검사 등을 수행하는 공무원이 관계인의 업무를 방해하거나 비밀을 누설할 경우에는 어떻게 되나요?',
            '위험물 사고 발생 시 사고의 원인과 피해를 조사하는 역할을 하는 공무원은 누구인가요?',
            '사고 조사를 위해 소방청장 등이 전문가로 이루어진 사고조사위원회를 구성할 수 있는데, 이를 누가 정하나요?',
            '탱크시험자의 업무를 적정하게 실시하도록 명령할 수 있는 공무원은 누구인가요?',
            '무허가장소에서 지정수량 이상의 위험물을 저장 또는 취급하는 경우, 어떤 조치를 할 수 있는가요?',
            '공공의 안전을 유지하기 위해 긴급 사용정지명령을 내릴 수 있는 공무원은 누구인가요?',
            '제조소등에서의 위험물의 저장 또는 취급이 규정에 위반될 경우, 어떤 조치를 할 수 있는가요?',

            '이동탱크저장소에서의 위험물의 저장 또는 취급이 규정에 위반될 경우, 어떤 조치를 할 수 있는가요?',   #130
            '이동탱크저장소의 관계인에게 조치를 명한 경우, 누구에게 그 내용을 통지해야 하나요?',
            '제조소등에서 위험물의 유출 등 사고가 발생했을 때, 관계인은 어떤 조치를 취해야 하나요?',
            '제조소등에서 사고를 발견한 경우, 어디에 통보해야 하나요?',
            '응급조치를 취하지 않는 경우, 어떤 조치를 취할 수 있나요?',
            '이동탱크저장소에서의 사고가 발생했을 때, 어떤 조치를 취할 수 있나요?',
            '위험물안전관리법에 따라 출입 및 검사 등을 수행하는 공무원은 어떤 증표를 지녀야 하나요?',
            '위험물안전관리법에 따라 출입 및 검사 등을 수행하는 공무원은 누구와 긴밀히 협력해야 하나요?',
            '소방청장의 역할은 무엇인가요?',
            '위험물 운반 차량을 정지시키는 권한을 가진 공무원은 누구인가요?',
            '위험물 저장 또는 취급장소의 관계인이 출입 및 검사 등의 권한을 행사하는 공무원의 신분을 확인할 때 요구할 수 있는 자격증은 무엇인가요?', #140


##################################################################<오상윤 Part3>###############################################################################################(141~209)
            '위험물 사고 발생 시 조사를 위해 구성되는 사고조사위원회의 구성은 어떻게 이루어지나요?',    #141
            '탱크시험자에게 필요한 명령을 내릴 수 있는 공무원은 누구인가요?',
            '무허가장소에서 지정수량 이상의 위험물을 저장 또는 취급하는 경우 어떤 조치를 할 수 있나요?',
            '위험물의 저장 또는 취급이 규정에 위반될 경우 어떤 조치를 할 수 있나요?',
            '위험물안전관리법에 따라 출입 및 검사 등을 수행하는 공무원은 어떤 증표를 지녀야 하나요?',
            '위험물안전관리법에 따라 출입 및 검사 등을 수행하는 공무원은 누구와 긴밀히 협력해야 하나요?',
            '공공의 안전을 유지하거나 재해의 발생을 방지하기 위해 제조소등의 사용을 일시정지하거나 제한하는 조치를 할 수 있는 공무원은 누구인가요?',
            '위험물 사고 발생 시 응급조치를 강구하지 않는 경우 어떤 조치를 취할 수 있나요?',
            '제조소등에서 위험물의 사고 발생 시 누구에게 통보해야 하나요?',
            '위험물의 안전관리와 관련된 업무를 수행하는 자는 어떤 교육을 받아야 하나요?',
            '제조소등의 관계인은 누구에게 안전교육을 받아야 하나요?',
            '안전교육의 과정 및 기간은 어떻게 정해지나요?',
            '안전교육을 받지 않은 교육대상자에 대해 어떤 조치를 취할 수 있나요?',
            '어떤 경우에는 처분을 하려면 청문을 실시해야 하나요?',
            '위험물 안전관리대행기관이 설립할 수 있는 협회의 역할은 무엇인가요?',
            '교육 대상자에게 수수료나 교육비를 납부해야 하는 경우는 어떤 경우인가요?',
            '어떤 경우에 소방청장 또는 시ㆍ도지사가 권한을 위임할 수 있나요?',
            '소방청장, 시ㆍ도지사, 소방본부장이 어떤 업무를 위탁할 수 있나요?',
            '공무원으로 취급되는 자에는 어떤 경우가 있나요?',
            '소방청장이나 시ㆍ도지사가 권한 일부를 위탁할 수 있는 대상은 누구인가요?',
            '소방기본법에 따라 업무를 위탁할 수 있는 대상은 누구인가요?',
            '어떤 경우에 수수료나 교육비를 납부하지 않아도 되나요?',
            '안전교육의 목적은 무엇인가요?',
            '협회의 설립 절차는 어떻게 되나요?',
            '협회가 소방청장의 인가를 받기 위한 절차는 어떻게 되나요?',
            '협회의 업무는 누가 정하나요?',
            '협회에 적용되는 법령은 무엇인가요?',
            '안전원이나 기술원이 위탁받은 업무를 수행하는 사람은 공무원으로 간주되나요?',
            '위험물의 안전관리자와 탱크시험자가 받아야 하는 교육은 어떤 것인가요?',
            '어떤 조직이 위험물의 안전관리와 사고 예방을 위해 협회를 설립할 수 있나요?',
            '협회 설립을 위한 인가 절차 및 정관의 기재사항은 어떻게 결정되나요?',
            '어떤 규정에 따라 교육 대상자가 수수료나 교육비를 납부해야 하나요?',
            '교육 대상자에게 수수료나 교육비를 납부해야 하는 경우를 제외한 예시는 무엇이 있나요?',
            '협회가 수행하는 업무에 대한 법적 근거는 어디에 명시되어 있나요?',
            '어떤 조직이 위험물의 안전관리와 관련된 업무를 위탁받아 수행할 수 있나요?',
            '위험물 저장 또는 취급 규정을 위반한 경우 어떤 처벌을 받을 수 있나요?',
            '업무상 과실로 규정을 위반한 경우 어떤 처벌을 받을 수 있나요?',
            '설치허가 없이 제조소를 설치한 경우 어떤 처벌을 받을 수 있나요?',
            '여러 가지 위반 사항에 대한 처벌은 어떻게 되나요?',
            '법인 대표자나 대리인이 위반 행위를 한 경우 어떤 처벌을 받나요?',
            '과태료 부과 대상에는 누가 포함되나요?',
            '과태료 부과의 절차는 어떻게 이루어지나요?',
            '과태료의 금액은 어떻게 결정되나요?',
            '어떤 경우에 업무상 과실로 처벌을 받을 수 있나요?',
            '위험물 운반에 관한 중요기준을 위반한 경우 어떤 처벌을 받을 수 있나요?',
            '안전관리자가 선임되지 않은 경우 어떤 처벌을 받을 수 있나요?',
            '위험물의 정기검사를 받지 않은 경우 어떤 처벌을 받을 수 있나요?',
            '안전조치 이행명령을 따르지 않은 경우 어떤 처벌을 받을 수 있나요?',
            '탱크시험자에 대한 감독상 명령에 따르지 않은 경우 어떤 처벌을 받을 수 있나요?',
            '무허가장소의 위험물에 대한 조치명령을 따르지 않은 경우 어떤 처벌을 받을 수 있나요?',
            '저장ㆍ취급기준 준수명령을 위반한 경우 어떤 처벌을 받을 수 있나요?',
            '위험물 저장소 운영자의 책임에 대해 어떻게 규정되어 있나요?',
            '안전관리에 대한 보고나 자료제출을 거부한 경우 어떤 처벌을 받을 수 있나요?',
            '어떤 조건을 위반했을 때 제조소등의 사용정지명령을 받을 수 있나요?',
            '안전조치 이행명령을 따르지 않아 벌금을 부과받게 되면 어디에 그 금액을 납부해야 하나요?',
            '제조소 설치허가를 받지 않고 제조소를 설치한 경우 어떤 처벌을 받을 수 있나요?',
            '정기점검을 하지 않거나 허위로 작성한 경우 어떤 처벌을 받을 수 있나요?',
            '안전관리자를 선임하지 않은 경우 어떤 처벌을 받을 수 있나요?',
            '저장ㆍ취급기준을 준수하지 않아 벌금을 부과받으면 누구에게 납부해야 하나요?',
            '제조소의 완공검사를 받지 않고 위험물을 저장ㆍ취급한 경우 어떤 처벌을 받을 수 있나요?',
            '변경허가를 받지 않고 제조소를 변경한 경우 어떤 처벌을 받을 수 있나요?',
            '탱크안전성능시험 또는 점검에 관한 업무를 허위로 한 경우 어떤 처벌을 받을 수 있나요?',
            '안전관리자나 그 대리자가 참여하지 않고 위험물을 취급한 경우 어떤 처벌을 받을 수 있나요?'    ]   #209


#답변리스트 작성(203개)

answers = [ '위험물안전관리법은 위험물의 저장, 취급 및 운반에 관련된 사항을 규정하며, 위험물로 인한 사고를 예방하여 공공의 안전을 확보하기 위해 제정되었습니다. 이 법은 항공기, 선박, 철도 및 궤도에 의한 위험물의 저장, 취급 및 운반에는 적용되지 않습니다.',
            '위험물은 인화성 또는 발화성 등의 성질을 가진 물품으로, 대통령령이 정하는 기준에 따라 분류됩니다.',
            '저장소는 지정수량 이상의 위험물을 저장하기 위한 장소이며, 취급소는 지정수량 이상의 위험물을 제조소가 아닌 장소에서 취급하기 위한 장소입니다.',
            '네, 지정수량 이상의 위험물은 저장소가 아닌 장소에서 저장하거나 제조소가 아닌 장소에서 취급할 수 없습니다. 다만, 시ㆍ도의 조례에 따라 특정한 조건 하에 임시로 저장 또는 취급할 수 있는 경우가 있습니다.',
            '제조소등에서의 위험물의 저장 및 취급에는 중요기준과 세부기준이 적용됩니다. 중요기준은 화재 등 위험의 예방과 응급조치에 영향을 미치는 기준으로, 행정안전부령에 의해 정의됩니다. 세부기준은 중요기준보다는 상대적으로 작은 영향을 미치는 기준으로, 행정안전부령에 따라 정의됩니다.',
            '위험물안전관리법은 공공의 안전을 확보하고 위험물로 인한 사고를 예방하기 위해 제정되었습니다. 위험물의 저장, 취급 및 운반은 공공의 안전과 직결되어 있으며, 이에 대한 적절한 규제와 안전대책이 필요하기 때문입니다.',
            '제조소등에서의 위험물의 저장 및 취급에는 중요기준과 세부기준이 적용됩니다. 중요기준은 화재 등 위험의 예방과 응급조치에 영향을 미치는 기준으로, 행정안전부령에 의해 정의됩니다. 세부기준은 중요기준보다는 상대적으로 작은 영향을 미치는 기준으로, 행정안전부령에 따라 정의됩니다.',
            '위험물안전관리법의 목적은 위험물로 인한 위해를 방지하여 공공의 안전을 확보하는 것입니다.',
            '제조소등의 위치, 구조, 설비에 관한 기술기준은 행정안전부령으로 정해집니다.',
            '국가는 위험물에 의한 사고를 예방하기 위해 다양한 시책을 수립ㆍ시행해야 합니다. 이에는 위험물의 유통실태 분석, 사고 유형의 분석, 안전기술 개발, 전문인력 양성 등이 포함됩니다. 또한, 지방자치단체의 위험물 관리를 지원하기 위한 행정적ㆍ재정적 지원도 이에 포함됩니다.',    #10

            '위험물안전관리법은 항공기, 선박, 철도 및 궤도에 의한 위험물의 저장, 취급 및 운반에는 적용되지 않습니다.',
            '위험물의 지정수량이란 대통령령에 의해 정하는 수량으로, 각 위험물의 종류에 따라 위험성을 고려하여 정해집니다. 이는 제조소나 저장소 등의 설치허가 등에 있어 최저 기준이 됩니다.',
            '위험물의 저장소는 지정수량 이상의 위험물을 저장하기 위한 장소로, 허가를 받은 장소를 의미합니다. 반면 취급소는 지정수량 이상의 위험물을 제조소가 아닌 장소에서 취급하는 곳으로, 역시 허가를 받은 장소입니다.',
            '네, 지정수량 이상의 위험물은 저장소가 아닌 장소에서 저장하거나 제조소가 아닌 장소에서 취급할 수 없습니다. 다만, 특정 조건을 충족하는 경우에는 시ㆍ도의 조례에 따라 임시로 저장 또는 취급이 가능할 수 있습니다.',
            '저장소 및 취급소에서의 안전관리에는 중요기준과 `세부기준이 적용됩니다. 중요기준은 화재 등 위험의 예방과 응급조치에 영향을 미치는 기준이며, 세부기준은 중요기준보다는 상대적으로 작은 영향을 미치는 기준입니다.',
            '국가는 위험물안전관리법에 따라 지방자치단체에게 위험물 사고 예방 및 대응을 위한 시책을 수립하고 이를 추진하기 위한 행정적ㆍ재정적 지원을 해야 합니다.',
            '제조소등의 위치, 구조, 설비에 대한 기술기준은 행정안전부령에 따라 정해집니다.',
            '위험물안전관리법은 공공의 안전을 확보하고 위험물로 인한 사고를 예방하여 공공의 안전을 증진시키는 것을 목적으로 합니다.',
            '위험물의 저장소는 지정수량 이상의 위험물을 저장하는 곳으로, 허가를 받은 장소입니다. 반면 취급소는 지정수량 이상의 위험물을 제조소가 아닌 장소에서 취급하는 곳으로, 역시 허가를 받은 장소입니다. 또한 취급소에서는 제조소와 달리 위험물을 제조하지 않습니다.',
            '국가는 위험물에 의한 사고를 예방하기 위해 위험물의 유통실태 분석, 사고 유형의 분석, 안전기술 개발, 전문인력 양성 등 다양한 사항을 수립ㆍ시행하여야 합니다. 또한 지방자치단체가 위험물 관리를 추진하는 데 필요한 행정적ㆍ재정적 지원을 제공해야 합니다.', #20

            '위험물 저장 또는 취급 시 출입 및 검사를 할 수 있는 권한은 소방청장, 시ㆍ도지사, 소방본부장, 소방서장에게 있습니다. (위험물안전관리법 제22조)',
            '주행 중인 위험물 운반 차량을 정지시켜 국가기술자격증 또는 교육수료증을 요구할 수 있는 권한은 소방공무원이나 경찰공무원에게 있습니다. (위험물안전관리법 제22조)',
            '소방본부장 또는 소방서장은 출입 및 검사 명령을 위험물을 저장 또는 취급하고 있다고 인정되는 장소의 관계인에게 내릴 수 있습니다. (위험물안전관리법 제22조)',
            '위험물 저장소에서 발생한 사고의 원인 및 피해를 조사하는 권한은 소방청장, 소방본부장, 소방서장에게 있습니다. (위험물안전관리법 제22조의2)',
            '소방공무원이나 경찰공무원이 국가기술자격증을 요구하고 이를 제시하지 않은 경우, 공무원은 주민등록증, 여권, 운전면허증 등의 신원확인을 요구하거나 신원확인을 위한 질문을 할 수 있습니다. (위험물안전관리법 제22조)',
            '출입 및 검사 명령을 받은 관계인이 거부한 경우, 공무원은 행정처분에 관한 법률에 따라 필요한 조치를 취할 수 있습니다. (위험물안전관리법 제22조)',
            '위험물 저장 또는 취급 시 출입을 제한하는 경우, 개인의 주거는 관계인의 승낙을 얻은 경우 또는 화재발생의 우려가 커서 긴급한 필요가 있는 경우가 아니면 출입할 수 없습니다. (위험물안전관리법 제22조)',
            '소방본부장 또는 소방서장이 출입 및 검사 명령을 내릴 수 있는 시간은 그 장소의 공개시간이나 근무시간내 또는 해가 뜬 후부터 해가 지기 전까지의 시간내에 행해야 합니다. (위험물안전관리법 제22조)',
            '출입 및 검사 명령을 수행하는 공무원은 그 권한을 표시하는 증표를 지니고 관계인에게 내보여야 합니다. (위험물안전관리법 제22조)',
            '위험물 저장소에서 발생한 사고의 조사를 위해 소방청장이 전문지식이 있는 사람으로 구성된 조사위원회를 둘 수 있는 권한은 소방청장에게 있습니다.',    #30

            '위험물시설의 설치를 위해 필요한 허가는 해당 시ㆍ도지사로부터 받아야 합니다.',
            '제조소등의 위치, 구조, 설비를 변경하려면 해당 시ㆍ도지사로부터 변경 허가를 받아야 합니다.',
            '제조소등의 위치, 구조, 설비 변경 시 변경 전후의 품명, 수량 등을 변경하고자 하는 날의 1일 전까지 해당 시ㆍ도지사에게 신고해야 합니다.',
            '제조소등에서 저장 또는 취급하는 위험물의 품명 및 수량을 변경하려면 변경하고자 하는 날의 1일 전까지 해당 시ㆍ도지사에게 신고해야 합니다.',
            '주택의 난방시설을 위한 저장소나 취급소, 농예용ㆍ축산용 또는 수산용으로 필요한 난방시설 등 지정수량 20배 이하의 저장소의 경우에는 제조소등의 설치나 변경에 대한 허가나 신고가 필요하지 않습니다.',
            '군부대를 위한 제조소등의 설치나 변경에는 군부대의 장과 해당 시ㆍ도지사 간의 협의가 필요하며, 이를 통해 허가를 받은 것으로 간주됩니다.',
            '위험물탱크의 설치나 변경공사를 진행할 때는 먼저 해당 시ㆍ도지사가 실시하는 탱크안전성능검사를 받아야 합니다. 이후 완공검사를 받기 전에는 제5조제4항의 기술기준에 적합한지 확인하는 절차가 필요합니다.',
            '제조소등의 설치나 변경을 마친 경우에는 시ㆍ도지사가 실시하는 완공검사를 받아야 합니다. 완공검사의 내용은 대통령령으로 정해지며, 기술기준에 적합하다고 인정받아야 해당 시설을 사용할 수 있습니다.',
            '제조소등의 설치자가 변경된 경우에는 해당 변경 사실을 시ㆍ도지사에게 신고해야 하며, 이 신고는 변경된 날부터 30일 이내에 이루어져야 합니다.',
            '제조소등의 사용을 중지하거나 중지한 후에는 해당 시ㆍ도지사에게 신고하여야 합니다. 또한, 사용 중지기간 동안에는 안전조치를 적합하게 해야 하며, 이에 대한 확인을 받아야 합니다.',    #40

            '제조소등 설치나 변경 후 완공검사를 받지 않은 경우에는 해당 시설을 사용할 수 없습니다. 따라서 변경 후에는 완공검사를 받고 적합하다고 인정받아야 합니다.',
            '제조소등의 설치자가 변경된 경우 변경 사실을 시ㆍ도지사에게 신고하는 기간은 변경된 날부터 30일 이내입니다.',
            '위험물안전관리법에 따른 과징금 부과의 기준은 사용의 정지가 이용자에게 심한 불편을 주거나 공익을 해칠 우려가 있을 때, 최대 2억원 이하의 과징금을 부과할 수 있다는 것입니다.',
            '제조소등의 사용을 중지하거나 재개하려면 해당 시ㆍ도지사에게 신고해야 합니다. 사용 중지나 재개하는 날의 14일 전까지 해당 신고를 완료해야 합니다.',
            '제조소등의 사용 중지나 재개에 따른 안전조치 확인은 시ㆍ도지사가 실시합니다. 해당 조치가 적합하게 이행되었는지를 확인하고 필요한 안전조치를 명령할 수 있습니다.',
            '제조소등의 사용 중지에 따른 위험물안전관리자 선임은 사용 중지 기간 동안에도 위험물안전관리자를 선임하지 않을 수 있습니다.',
            '제조소등 설치허가 취소나 사용정지에 따른 과징금 부과는 해당 조치가 이용자에게 심한 불편을 주거나 공익을 해칠 우려가 있는 경우에 이루어집니다.',
            '제조소등의 용도 폐지 시 해당 시ㆍ도지사에게 신고해야 합니다. 용도 폐지가 완료된 날부터 14일 이내에 해당 신고를 완료해야 합니다.',
            '제조소등의 사용 중지 또는 재개 시 시ㆍ도지사는 해당 조치에 따라 안전조치를 확인하고 필요한 경우 위해 방지를 위한 명령을 내릴 수 있습니다.',
            '제조소등 설치허가의 취소나 사용정지에 따라 과징금이 부과되는 경우 「지방행정제재ㆍ부과금의 징수 등에 관한 법률」에 따라 징수됩니다.',  #50

            '위험물시설을 설치하려는 경우에는 해당 지역의 특별시장, 광역시장, 특별자치시장, 도지사 등의 허가를 받아야 합니다. (위험물안전관리법 제6조)',
            '제조소등에서 저장하거나 취급하는 위험물의 품명ㆍ수량 변경을 하려는 경우, 변경하고자 하는 날의 1일 전까지 관할 시ㆍ도지사에게 신고해야 합니다. (위험물안전관리법 제6조 제2항)',
            '군부대가 제조소등을 설치하거나 변경하기 위해 군부대의 장은 해당 제조소등의 소재지를 관할하는 시ㆍ도지사와 협의해야 합니다. (위험물안전관리법 제7조)',
            '제조소등에서 위험물탱크의 설치 또는 변경공사를 할 때는 시ㆍ도지사가 실시하는 탱크안전성능검사를 받아야 합니다. (위험물안전관리법 제8조)',
            '제조소등의 설치자가 사망하거나 양도/인도한 경우, 상속인, 양수ㆍ인수한 자 또는 합병 후 존속하는 법인이 그 지위를 승계합니다. (위험물안전관리법 제10조)',
            '제조소등의 용도를 폐지하기 위해서는 해당 시ㆍ도지사에게 신고해야 합니다. (위험물안전관리법 제11조)',
            '제조소등의 사용을 중지하려는 경우에는 위험물의 제거 및 출입통제 등 안전조치를 행해야 합니다. (위험물안전관리법 제11조의2)',
            '제조소등의 위치나 구조를 변경하거나 완공검사를 받지 않고 사용하는 등의 위반사항이 있을 경우, 사용 정지나 허가 취소가 될 수 있습니다. (위험물안전관리법 제12조)',
            '제조소등의 관계인이 안전조치를 적절히 하지 않을 경우, 사용 정지나 과징금 부과 등의 조치가 이루어질 수 있습니다. (위험물안전관리법 제13조)',
            '안전관리자는 제조소 및 이동탱크저장소에서 위험물의 안전관리에 관한 직무를 담당하며, 이를 위해 필요한 자격을 갖추어야 합니다.', #60

            '안전관리자를 선임하기 위해서는 대통령령이 정하는 위험물취급자격자로 선임해야 합니다.',
            '안전관리자를 선임한 후에는 해임이나 퇴직 시 30일 이내에 재선임을 해야 하며, 그 외에도 안전관리자의 변경이 있을 경우 14일 이내에 관련 기관에 신고해야 합니다.',
            '안전관리자는 위험물을 다루는 작업에 안전하게 관리하고 감독하여 안전사고를 예방하는 역할을 하기 때문에 매우 중요합니다.',
            '안전관리자는 위험물을 다루는 작업을 할 때 작업자에게 안전에 관련된 지시를 내려야 하고, 해당 시설의 관계자들은 안전관리자의 지시를 따라야 합니다.',
            '안전관리자의 역할은 위험물을 다루는 모든 제조소 및 이동탱크저장소에서 중요하며, 특히 다량의 위험물을 다루는 업종에서는 그 역할이 더욱 중요합니다.',
            '안전관리자가 일시적으로 업무를 수행할 수 없는 경우, 대리자를 지정하여 대행하게 됩니다. 대리자는 해당 업무를 최대 30일 동안 대행할 수 있습니다.',
            '안전관리자의 업무에 대해 의견을 제기할 수 있는 경우는 대통령령이나 행정안전부령에 따라 안전에 관한 권고사항을 준수하지 않거나 위반했을 때입니다.',
            '안전관리자가 여행이나 질병 등으로 업무를 수행할 수 없는 경우, 대통령령이나 행정안전부령에 따라 안전에 관한 자격을 갖춘 다른 자를 대리자로 지정하여 업무를 대행하게 됩니다.',
            '다수의 제조소에서 하나의 안전관리자를 선임하는 경우, 대통령령이 정하는 자격을 갖춘 대리자를 각 제조소별로 선정하여 안전관리자를 보조하도록 조치를 취해야 합니다. 이를 통해 안전관리자가 효율적으로 업무를 수행할 수 있습니다.',#
            '안전관리자는 위험물을 다루는 환경에서 작업자의 안전을 보장하고 사고 예방을 위해 책임을 집니다.',   #70

            '안전관리자는 대통령령이 정하는 위험물 취급자격자로 선임되어야 하며, 해당 업무에 필요한 전문적인 지식과 능력을 갖추어야 합니다.',
            '안전관리자는 제조소나 저장소에서 위험물의 취급, 저장, 운반 등을 포함한 안전 관리 업무를 담당합니다. 또한 사고 예방 및 대응, 안전 교육 등 다양한 역할을 수행합니다.',
            '안전관리자는 화학물질관리법, 위험물안전관리법 등과 같은 관련 법규에 따라 업무를 수행해야 합니다. 또한 해당 업무에 관련된 규정과 지침을 준수해야 합니다.',
            '안전관리자는 시설과 장비의 안전 상태를 점검하고, 위험을 예방하기 위해 작업자들에게 안전 교육 및 훈련을 제공해야 합니다. 또한 안전 시설 및 장비의 유지보수를 감독하고 사고 발생 시 신속하게 대응해야 합니다.',
            '안전관리자는 안전에 관한 교육을 받고, 해당 분야에서의 경험이 있어야 합니다. 또한 안전에 대한 이해도와 리더십 능력을 갖추어야 합니다.',
            '안전관리자는 위험물을 다루는 환경에서 작업자의 생명과 안전을 지키는 중요한 역할을 담당합니다. 그들의 업무는 사고 예방과 대응을 통해 작업환경의 안전성을 유지하는 데 큰 영향을 미칩니다.',
            '안전관리자는 작업환경에서 발생할 수 있는 위험을 분석하고, 그에 따른 대응책을 마련해야 합니다. 이는 작업자의 안전을 보장하고 사고를 최소화하기 위함입니다.',
            '안전관리자는 작업자에게 화학물질의 위험성과 안전 조치에 대한 교육을 제공해야 합니다. 또한 작업환경에서의 안전수칙 및 사고 대응 절차 등을 교육하여 작업자들의 안전을 증진시켜야 합니다.',
            '안전관리자는 작업환경에서 발생할 수 있는 위험을 예측하고, 이에 대한 대비책을 마련하여 작업자들의 안전을 보장해야 합니다. 또한 정기적인 안전점검 및 교육 프로그램을 운영하여 작업환경을 안전하게 유지해야 합니다.',
            '위험물안전관리법(위험물안전관리 및 안전보고에 관한 법률)은 위험물 시설의 안전관리에 관한 사항을 규정하고 있습니다. (제1조)',   #80

            '제15조는 제조소등의 관계인이 안전관리자를 선임해야 함을 규정하고 있습니다.',
            '안전관리자는 시설 내에서 위험물의 안전관리에 관한 업무를 수행합니다.',
            '안전관리자를 선임하지 않으면 시설의 안전관리가 제대로 이루어지지 않을 수 있습니다.',
            '대통령령이 정하는 위험물취급자격자를 안전관리자로 선임해야 합니다.',
            '안전관리자의 선임과 해임에 대한 절차 및 기간이 규정되어 있습니다.',
            '안전관리자가 일시적으로 직무를 수행할 수 없는 경우 대리자를 지정하여 직무를 대행할 수 있습니다.',
            '안전관리자는 작업자에게 안전관리에 관한 필요한 지시를 하고, 위험물의 취급에 관한 안전관리와 감독을 합니다.',
            '안전관리자가 선임되지 않은 경우에는 관련 법령에 따라 안전관리자를 선임하여야 합니다.',
            '제조소등의 관계인과 종사자는 안전관리자의 의견을 존중하고 그 권고에 따라야 합니다.',
            '위험물의 운반은 용기ㆍ적재방법 및 운반방법에 관한 중요기준과 세부기준에 따라 행해져야 합니다.',    #90

            '위험물을 운반하는 차량의 운전자는 국가기술자격법에 따른 위험물 분야의 자격을 취득하거나 제28조 제1항에 따른 교육을 수료해야 합니다.',
            '시ㆍ도지사는 운반용기를 제작하거나 수입한 자의 신청에 따라 운반용기를 검사할 수 있습니다. 단, 특정한 조건에 해당하는 경우 추가적인 검사가 요구됩니다.',
            '위험물운송자는 행정안전부령으로 정하는 기준을 준수하여 위험물의 안전확보를 위해 세심한 주의를 기울여야 합니다.',
            '운송책임자는 위험물 운송의 감독 또는 지원을 담당하며, 운송책임자의 범위와 감독 또는 지원 방법은 행정안전부령으로 구체적으로 정해집니다.',
            '이동탱크저장소에 의한 위험물 운송 시에는 제20조 제2항의 요건을 준수해야 하며, 대통령령이 정하는 운송책임자의 감독 또는 지원을 받아야 합니다.',
            '위험물의 운반 중 안전사고 발생 시에는 즉시 응급조치를 취하고 관련 당국에 신고해야 합니다. 또한 사고 조사와 사고 예방을 위한 조치를 취해야 합니다.',
            '위험물운송자는 국가기술자격법에 따른 위험물 분야의 자격을 취득하거나 관련 교육을 이수함으로써 자격을 취득할 수 있습니다.',
            '운송책임자의 감독 또는 지원 방법은 행정안전부령으로 구체적으로 정해지며, 이에 따라 운송책임자는 적절한 감독 또는 지원을 받아야 합니다.',
            '이동탱크저장소에 의한 위험물 운송 시에는 행정안전부령에 따른 운송용기의 규격과 안전관리에 관한 내용이 적용됩니다.',
            '위험물 운반 시 중요한 기준은 화재 등 위험의 예방과 응급조치에 큰 영향을 미치는 기준으로, 행정안전부령에 의해 정해집니다.', #100

            '위험물의 운반용기에 대한 검사를 받지 않은 경우, 해당 운반용기는 사용이 허가되지 않을 수 있으며, 법적인 제재를 받을 수도 있습니다.',
            '위험물을 운송하는 운송책임자의 주요 역할은 운송의 안전성을 확보하기 위해 운송과정을 감독하고 안전 조치를 취하는 것입니다.',
            '위험물의 운반 중 발생할 수 있는 안전사고에는 화재, 폭발, 유출 등이 있으며, 이로 인해 환경 오염 및 인명피해 등이 발생할 수 있습니다.',
            '위험물 운송책임자의 감독 또는 지원을 받는 방법은 해당 업체가 소속된 지방 관할 당국과의 협조를 통해 이루어집니다.',
            '운송책임자의 감독 또는 지원을 받는 목적은 운송과정에서의 안전을 보장하고 사고 발생 시 신속한 대응을 위함입니다.',
            '위험물 운송 중 화재 발생 시에는 즉시 주변인들을 안전한 장소로 이동시키고 화재 진압을 위한 응급조치를 취해야 합니다.',
            '운송용기에 대한 검사는 시ㆍ도지사가 실시하며, 해당 검사를 통과한 운송용기만이 안전하게 운송에 사용될 수 있습니다.',
            '위험물 운송 시에는 운송용기의 안전한 조작 및 보관, 안전장비의 사용 등 특별한 주의사항을 준수해야 합니다.',
            '운송책임자가 운송 중 사고 발생 시에는 해당 사고와 관련된 법적인 책임을 지게 될 수 있으며, 이에 따른 추가 조치가 요구될 수 있습니다.',

            '위험물의 운반은 그 용기ㆍ적재방법 및 운반방법에 관한 중요기준과 세부기준에 따라 행해져야 합니다. 중요기준은 화재 등 위해의 예방과 응급조치에 큰 영향을 미치거나 그 기준을 위반하는 경우 직접적으로 화재를 일으킬 가능성이 큰 기준입니다(위험물안전관리법 제20조)', #110
            '위험물의 운송책임자는 운송과정에서의 안전을 보장하기 위해 운송의 감독 또는 지원을 담당합니다. 이를 위해 필요한 법령적 근거는 행정안전부령에 명시되어 있습니다(위험물안전관리법 제21조)',
            '위험물의 운반 중에는 화재, 폭발, 유출 등의 안전사고가 발생할 수 있습니다. 이러한 안전사고는 환경 오염 및 인명피해 등을 초래할 수 있으므로, 운반 과정에서의 안전관리가 중요합니다(위험물안전관리법 제21조)',
            '운송용기에 대한 검사는 시ㆍ도지사에 의해 실시되며, 해당 검사를 통과한 운송용기만이 안전하게 운송에 사용될 수 있습니다. 이는 행정안전부령에 근거하여 시행됩니다(위험물안전관리법 제20조)',
            '위험물 운송 중 화재 발생 시에는 즉시 주변인들의 안전을 보장하고, 화재 진압을 위한 응급조치를 취해야 합니다. 이는 행정안전부령에 명시된 안전확보 기준에 따라 수행되어야 합니다(위험물안전관리법 제20조)',
            '위험물을 운반하는 운송책임자는 국가기술자격법에 따른 위험물 분야의 자격을 취득하거나, 교육을 수료하여야 합니다(위험물안전관리법 제20조)',
            '운송책임자의 감독 또는 지원은 운송과정에서의 안전을 보장하고, 사고 발생 시 신속한 대응을 위한 것입니다. 이를 위해 구체적인 기준은 행정안전부령에 의해 정해지고 있습니다(위험물안전관리법 제21조)',
            '운송 중에는 운송용기의 안전한 조작 및 보관, 안전장비의 사용 등 세심한 주의가 요구됩니다. 이는 행정안전부령에 근거하여 수행되어야 합니다(위험물안전관리법 제21조)',
            '운송용기의 검사를 받지 않은 경우, 해당 운송용기는 사용이 허가되지 않을 수 있으며, 법적인 제재를 받을 수도 있습니다. 이는 위험물안전관리법에 근거하여 시행됩니다(위험물안전관리법 제20조)',
            '위험물 운송책임자의 감독 또는 지원은 해당 업체가 소속된 지방 관할 당국과의 협조를 통해 이루어집니다. 이에 대한 구체적인 기준은 행정안전부령에 명시되어 있습니다(위험물안전관리법 제21조)', #120

            '위험물의 저장 또는 취급장소에 출입하여 검사하거나 자료를 요구할 수 있는 공무원은 소방청장, 시ㆍ도지사, 소방본부장, 소방서장입니다.',
            '위험물 운반 차량을 정지시켜 운반자의 자격을 확인하는 권한을 가진 공무원은 소방공무원 또는 경찰공무원입니다.',
            '출입 및 검사 등의 규정은 해당 장소의 공개시간이나 근무시간 내, 또는 해가 뜬 후부터 해가 지기 전까지의 시간 내에 행해져야 합니다.',
            '출입 및 검사 등을 수행하는 공무원이 관계인의 업무를 방해하거나 비밀을 누설할 경우, 그러한 행위는 금지되며, 엄격히 제재될 수 있습니다.',
            '위험물 사고 발생 시 사고의 원인과 피해를 조사하는 역할을 하는 공무원은 소방청장, 소방본부장, 소방서장입니다.',
            '사고 조사를 위해 소방청장 등이 전문가로 이루어진 사고조사위원회를 구성할 때에는 그 구성 및 운영 등에 관한 사항은 대통령령으로 정해집니다.',
            '탱크시험자의 업무를 적정하게 실시하도록 명령할 수 있는 공무원은 시ㆍ도지사, 소방본부장, 소방서장입니다.',
            '무허가장소에서 지정수량 이상의 위험물을 저장 또는 취급하는 경우, 시ㆍ도지사, 소방본부장, 소방서장은 그 위험물 및 시설의 제거 등 필요한 조치를 명할 수 있습니다.',
            '공공의 안전을 유지하기 위해 긴급 사용정지명령을 내릴 수 있는 공무원은 시ㆍ도지사, 소방본부장, 소방서장입니다.',
            '제조소등에서의 위험물의 저장 또는 취급이 규정에 위반될 경우, 시ㆍ도지사, 소방본부장, 소방서장은 해당 제조소등의 관계인에게 동항의 기준에 따라 조치를 명할 수 있습니다.',   #130

            '이동탱크저장소에서의 위험물의 저장 또는 취급이 규정에 위반될 경우, 시ㆍ도지사, 소방본부장, 소방서장은 해당 이동탱크저장소의 관계인에게 동항의 기준에 따라 조치를 명할 수 있습니다.',
            '이동탱크저장소의 관계인에게 조치를 명한 경우, 행정안전부령에 따라 시ㆍ도지사, 소방본부장, 소방서장이 그 내용을 통지해야 합니다.',
            '제조소등에서 위험물의 유출 등 사고가 발생했을 때, 관계인은 즉시 위험물의 유출 및 확산 방지, 유출된 위험물의 제거 등의 응급조치를 취해야 합니다.',
            '제조소등에서 사고를 발견한 경우, 즉시 소방서, 경찰서 또는 관계기관에 통보해야 합니다.',
            '응급조치를 취하지 않는 경우, 소방본부장 또는 소방서장은 해당 제조소등의 관계인에게 응급조치를 취하도록 명할 수 있습니다.',
            '이동탱크저장소에서의 사고가 발생했을 때, 소방본부장 또는 소방서장은 해당 이동탱크저장소의 관계인에게 응급조치를 취하도록 명할 수 있습니다.',
            '위험물안전관리법에 따라 출입 및 검사 등을 수행하는 공무원은 그 권한을 표시하는 증표를 지니고 관계인에게 내보여야 합니다.',
            '위험물안전관리법에 따라 출입 및 검사 등을 수행하는 공무원은 소방공무원과 경찰공무원과 긴밀히 협력해야 합니다.',
            '소방청장은 위험물의 저장 또는 취급에 따른 화재의 예방 및 진압대책을 수립하고 관리하는 역할을 담당합니다. (위험물안전관리법 제22조)',
            '위험물 운반 차량을 정지시키는 권한을 가진 공무원은 소방공무원 또는 경찰공무원입니다. (위험물안전관리법 제22조)',   #140

            '위험물 저장 또는 취급장소의 관계인이 출입 및 검사 등의 권한을 행사하는 공무원의 신분을 확인할 때 요구할 수 있는 자격증은 국가기술자격증 또는 교육수료증입니다. (위험물안전관리법 제22조)',
            '위험물 사고 발생 시 조사를 위해 구성되는 사고조사위원회는 소방청장 등이 전문지식이 있는 사람으로 구성됩니다. (위험물안전관리법 제22조의2)',
            '탱크시험자에게 필요한 명령을 내릴 수 있는 공무원은 시ㆍ도지사, 소방본부장, 소방서장입니다. (위험물안전관리법 제23조)',
            '무허가장소에서 지정수량 이상의 위험물을 저장 또는 취급하는 경우, 시ㆍ도지사, 소방본부장, 소방서장은 해당 위험물 및 시설의 제거 등 필요한 조치를 명할 수 있습니다. (위험물안전관리법 제24조)',
            '위험물의 저장 또는 취급이 규정에 위반될 경우, 시ㆍ도지사, 소방본부장, 소방서장은 해당 제조소등의 관계인에게 조치를 명할 수 있습니다. (위험물안전관리법 제26조)',
            '위험물안전관리법에 따라 출입 및 검사 등을 수행하는 공무원은 그 권한을 표시하는 증표를 지니고 관계인에게 내보여야 합니다. (위험물안전관리법 제22조)',
            '위험물안전관리법에 따라 출입 및 검사 등을 수행하는 공무원은 소방공무원과 경찰공무원과 긴밀히 협력해야 합니다. (위험물안전관리법 제22조)',
            '공공의 안전을 유지하거나 재해의 발생을 방지하기 위해 제조소등의 사용을 일시정지하거나 제한하는 조치를 할 수 있는 공무원은 시ㆍ도지사, 소방본부장, 소방서장입니다. (위험물안전관리법 제25조)',
            '위험물 사고 발생 시 응급조치를 강구하지 않는 경우, 소방본부장 또는 소방서장은 응급조치를 강구하도록 명할 수 있습니다. (위험물안전관리법 제27조)',
            '제조소등에서 위험물의 사고 발생 시, 그 사실을 즉시 소방서, 경찰서 또는 관계기관에 통보해야 합니다. (위험물안전관리법 제27조)', #150

            '대통령령이 정하는 안전관리자, 탱크시험자, 위험물운반자, 위험물운송자 등 해당 업무에 관한 능력의 습득 또는 향상을 위한 소방청장이 실시하는 교육을 받아야 합니다.',
            '제조소등의 관계인은 정부의 지침에 따라 제1항의 규정에 따른 교육대상자에 대하여 필요한 안전교육을 받게 하여야 합니다.',
            '행정안전부령으로 정해집니다.',
            '해당 업무를 제한할 수 있습니다.',
            '제12조의 규정에 따른 제조소등 설치허가의 취소 또는 제16조제5항의 규정에 따른 탱크시험자의 등록취소와 같은 경우에는 청문을 실시해야 합니다.',
            '위험물 안전관리대행기관이 설립할 수 있는 협회는 위험물의 안전관리, 사고 예방을 위한 안전기술 개발 등을 위해 설립되며, 위험물 안전관리의 건전한 발전을 도모합니다.',
            '제28조에서 정한 교육 대상자가 승인, 허가, 검사, 교육 등을 받거나 등록, 신고를 하려면 행정안전부령으로 정하는 바에 따라 수수료나 교육비를 납부해야 합니다.',
            '소방청장 또는 시ㆍ도지사는 이 법에 따른 권한의 일부를 대통령령이 정하는 바에 따라 시ㆍ도지사, 소방본부장 또는 소방서장에게 위임할 수 있습니다.',
            '소방청장, 시ㆍ도지사, 소방본부장은 이 법에 따른 업무의 일부를 대통령령이 정하는 바에 따라 소방기본법 제40조의 규정에 의한 한국소방안전원 또는 기술원에 위탁할 수 있습니다.',
            '형법 제129조 내지 제132조의 적용에 있어서는 형법 제129조 내지 제132조의 적용에 있어서 공무원으로 봅니다. 그 예로는 제8조제1항 후단의 규정에 따른 검사업무에 종사하는 기술원의 담당 임원 및 직원, 제16조제1항의 규정에 따른 탱크시험자의 업무에 종사하는 자, 제30조제2항의 규정에 따라 위탁받은 업무에 종사하는 안전원 및 기술원의 담당 임원 및 직원 등이 있습니다.',   #160

            '소방청장이나 시ㆍ도지사는 이 법에 따른 권한의 일부를 대통령령이 정하는 바에 따라 시ㆍ도지사, 소방본부장 또는 소방서장에게 위임할 수 있습니다.',
            '소방청장, 시ㆍ도지사, 소방본부장 또는 소방서장은 이 법에 따른 업무의 일부를 대통령령이 정하는 바에 따라 소방기본법 제40조의 규정에 의한 한국소방안전원 또는 기술원에 위탁할 수 있습니다.',
            '행정안전부령으로 정하는 바에 따라 수수료나 교육비를 납부하지 않아도 되는 경우도 있습니다.',
            '제28조에서 규정된 안전교육의 목적은 해당 업무에 관한 능력의 습득 또는 향상을 위함입니다.',
            '협회는 법인으로 하며, 소방청장의 인가를 받아 주된 사무소의 소재지에 설립등기를 함으로써 성립합니다.',
            '협회의 설립인가 절차 및 정관의 기재사항 등에 관한 사항은 대통령령으로 정해집니다.',
            '협회의 업무는 정관으로 정합니다.',
            '협회에 관하여 이 법에서 규정한 것 외에는 민법 중 사단법인에 관한 규정을 준용합니다.',
            '네, 안전원이나 기술원이 위탁받은 업무를 수행하는 사람은 형법 제129조 내지 제132조의 적용에 있어서는 공무원으로 간주됩니다.',
            '위험물의 안전관리자, 탱크시험자 등 해당 업무를 수행하는 자는 대통령령이 정하는 안전교육을 받아야 합니다. (위험물안전관리법 제28조)',   #170

            '제조소등의 관계인, 위험물운송자, 탱크시험자 및 안전관리자의 업무를 수행하는 안전관리대행기관이 협회를 설립할 수 있습니다. (위험물안전관리법 제29조의2)',
            '협회의 설립인가 절차 및 정관의 기재사항 등에 관한 사항은 대통령령으로 결정됩니다. (위험물안전관리법 제29조의2)',
            '제28조에서 정한 교육 대상자가 교육 등을 받거나 등록, 신고를 하려면 행정안전부령으로 정하는 바에 따라 수수료나 교육비를 납부해야 합니다. (위험물안전관리법 제31조)',
            '교육 대상자가 수수료나 교육비를 납부하지 않아도 되는 경우는 행정안전부령으로 정해진 예외 사항이 있습니다. (위험물안전관리법 제31조)',
            '협회가 수행하는 업무의 법적 근거는 위험물안전관리법에서 명시되어 있습니다. (위험물안전관리법)',
            '안전관리대행기관이 위험물의 안전관리와 관련된 업무를 위탁받아 수행할 수 있습니다. (위험물안전관리법 제29조의2)',
            '저장 또는 취급 규정을 위반하면 1년 이상 10년 이하의 징역이나 벌금이 부과될 수 있습니다.',
            '업무상 과실로 규정을 위반하면 7년 이하의 금고 또는 7천만원 이하의 벌금이 부과될 수 있습니다.',
            '설치허가 없이 제조소를 설치하면 5년 이하의 징역 또는 1억원 이하의 벌금이 부과될 수 있습니다.',
            '다양한 위반 사항에 대해 각각 다른 벌칙이 부과됩니다.', #180

            '법인 대표자나 대리인이 위반 행위를 한 경우, 그 법인 또는 개인에게도 해당 조문의 벌금형이 부과됩니다.',
            '과태료는 다양한 위반 행위에 대해 부과되며, 승인을 받지 않은 자, 변경신고를 하지 않은 자 등이 해당됩니다.',
            '과태료 부과는 시ㆍ도지사, 소방본부장 또는 소방서장이 부과하고 징수합니다.',
            '과태료의 금액은 각각의 위반 행위에 따라 다르며, 대통령령에 따라 정해집니다.',
            '업무상 과실로 처벌을 받는 경우에는 해당 업무를 수행함에 있어서 상당한 주의와 관리를 하지 않아 발생한 위험하고 예방 가능한 사고가 발생한 경우입니다.',
            '위험물 운반에 관한 중요기준을 위반하면 1천만원 이하의 벌금이 부과될 수 있습니다.',
            '안전관리자가 선임되지 않은 경우, 관련 법령에 따라 1천만원 이하의 벌금이 부과될 수 있습니다.',
            '위험물의 정기검사를 받지 않은 경우, 3천만원 이하의 벌금이 부과될 수 있습니다.',
            '안전조치 이행명령을 따르지 않은 경우, 해당 법령에 따라 1천만원 이하의 벌금이 부과될 수 있습니다.',
            '탱크시험자에 대한 감독상 명령에 따르지 않은 경우, 해당 법령에 따라 1천만원 이하의 벌금이 부과될 수 있습니다.', #190

            '무허가장소의 위험물에 대한 조치명령을 따르지 않은 경우, 1천만원 이하의 벌금이 부과될 수 있습니다.',
            '저장ㆍ취급기준 준수명령을 위반한 경우, 해당 법령에 따라 1천만원 이하의 벌금이 부과될 수 있습니다.',
            '위험물 저장소 운영자는 안전관리에 책임이 있으며, 관련 법령을 준수해야 합니다. 위반 시에는 법에 따라 적절한 처벌이 가해질 수 있습니다.',
            '안전관리에 대한 보고나 자료제출을 거부한 경우, 해당 법령에 따라 1천만원 이하의 벌금이 부과될 수 있습니다.',
            '위험물안전관리법에 명시된 이행명령, 정기점검을 하지 않은 경우 사용정지명령을 받을 수 있습니다.',
            '행정안전부령이 정하는 바에 따라 정부에 수수료를 납부해야 합니다. (위험물안전관리법 제39조 제2항)',
            '제조소 설치허가를 받지 않고 제조소를 설치한 경우, 해당 조항에 따라 5년 이하의 징역 또는 1억원 이하의 벌금에 처할 수 있습니다. (위험물안전관리법 제34조의2)',
            '정기점검을 하지 않거나 허위로 작성한 경우, 7년 이하의 금고 또는 7천만원 이하의 벌금에 처할 수 있습니다. (위험물안전관리법 제34조)',
            '안전관리자를 선임하지 않은 경우, 해당 조항에 따라 1천만원 이하의 벌금이 부과될 수 있습니다. (위험물안전관리법 제35조)',
            '저장ㆍ취급기준을 준수하지 않아 벌금을 부과받게 되면 해당 조항에 따라 시ㆍ도지사, 소방본부장 또는 소방서장이 부과하고 징수합니다. (위험물안전관리법 제39조 제2항)', #200

            '제조소의 완공검사를 받지 않고 위험물을 저장ㆍ취급한 경우, 해당 조항에 따라 3년 이하의 징역 또는 3천만원 이하의 벌금에 처할 수 있습니다. (위험물안전관리법 제34조의3)',
            '변경허가를 받지 않고 제조소를 변경한 경우, 해당 조항에 따라 1천500만원 이하의 벌금에 처할 수 있습니다. (위험물안전관리법 제36조)',
            '탱크안전성능시험 또는 점검에 관한 업무를 허위로 한 경우, 해당 조항에 따라 1천만원 이하의 벌금에 처할 수 있습니다. (위험물안전관리법 제36조)',
            '안전관리자나 그 대리자가 참여하지 않고 위험물을 취급한 경우, 해당 조항에 따라 1천만원 이하의 벌금에 처할 수 있습니다. (위험물안전관리법 제37조)'   ]

#### 데이터 전처리.

In [6]:
# 태거 객체 생성.
tagger = Twitter()

# 새로운 단어와 해당 품사 추가.(명사 182개)
my_new_words =      [   ('저장소', 'Noun'),
                        ('취급소', 'Noun'),
                        ('제조소', 'Noun'),
                        ('위치', 'Noun'),
                        ('구조', 'Noun'),
                        ('설비', 'Noun'),
                        ('기술기준', 'Noun'),
                        ('대상', 'Noun'),
                        ('지정수량', 'Noun'),
                        ('구분', 'Noun'),
                        ('출입', 'Noun'),
                        ('검사', 'Noun'),
                        ('권한', 'Noun'),
                        ('사고', 'Noun'),
                        ('피해', 'Noun'),
                        ('조사', 'Noun'),
                        ('소방청장', 'Noun'),
                        ('군부대', 'Noun'),
                        ('조사위원회', 'Noun'),
                        ('설치자', 'Noun'),
                        ('완공검사', 'Noun'),
                        ('사유', 'Noun'),
                        ('법률', 'Noun'),
                        ('허가', 'Noun'),
                        ('조치', 'Noun'),
                        ('안전조치', 'Noun'),
                        ('과징금', 'Noun'),
                        ('책임', 'Noun'),
                        ('자격', 'Noun'),
                        ('역할', 'Noun'),
                        ('위험 분석', 'Noun'),
                        ('교육', 'Noun'),
                        ('환경', 'Noun'),
                        ('규정', 'Noun'),
                        ('내용', 'Noun'),
                        ('문제', 'Noun'),
                        ('기준', 'Noun'),
                        ('해임', 'Noun'),
                        ('대행', 'Noun'),
                        ('수행', 'Noun'),
                        ('처리', 'Noun'),
                        ('존중', 'Noun'),
                        ('요건', 'Noun'),
                        ('감독', 'Noun'),
                        ('참여', 'Noun'),
                        ('이동탱크저장소', 'Noun'),
                        ('행정안전부령', 'Noun'),
                        ('운송', 'Noun'),
                        ('시', 'Noun'),
                        ('운반용기', 'Noun'),
                        ('운송책임자', 'Noun'),
                        ('안전사고', 'Noun'),
                        ('종류', 'Noun'),
                        ('지원', 'Noun'),
                        ('방법', 'Noun'),
                        ('목적', 'Noun'),
                        ('대응', 'Noun'),
                        ('절차', 'Noun'),
                        ('용기', 'Noun'),
                        ('주의사항', 'Noun'),
                        ('발생', 'Noun'),
                        ('범위', 'Noun'),
                        ('안전', 'Noun'),
                        ('주의', 'Noun'),
                        ('공무원', 'Noun'),
                        ('공공', 'Noun'),
                        ('사용정지명령', 'Noun'),
                        ('위반', 'Noun'),
                        ('원인', 'Noun'),
                        ('사고조사위원회', 'Noun'),
                        ('구성', 'Noun'),
                        ('탱크시험자', 'Noun'),
                        ('명령', 'Noun'),
                        ('무허가장소', 'Noun'),
                        ('저장', 'Noun'),
                        ('취급', 'Noun'),
                        ('소', 'Noun'),
                        ('안전관리자', 'Noun'),
                        ('선임', 'Noun'),
                        ('처벌', 'Noun'),
                        ('취급기준', 'Noun'),
                        ('벌금', 'Noun'),
                        ('납부', 'Noun'),
                        ('변경허가', 'Noun'),
                        ('변경', 'Noun'),
                        ('탱크안전성능시험', 'Noun'),
                        ('점검', 'Noun'),
                        ('소방공무원', 'Noun'),
                        ('경찰공무원', 'Noun'),
                        ('위험물안전관리법', 'Noun'),
                        ('소방본부장', 'Noun'),
                        ('소방서장', 'Noun'),
                        ('신원확인', 'Noun'),
                        ('공개시간', 'Noun'),
                        ('관계인', 'Noun'),
                        ('증표', 'Noun'),
                        ('시간', 'Noun'),
                        ('변경공사', 'Noun'),
                        ('사망', 'Noun'),
                        ('양도', 'Noun'),
                        ('사용정지', 'Noun'),
                        ('용도', 'Noun'),
                        ('관계자', 'Noun'),
                        ('대리자', 'Noun'),
                        ('응급', 'Noun'),
                        ('출입통제', 'Noun'),
                        ('신고', 'Noun'),
                        ('소재지', 'Noun'),
                        ('재선임', 'Noun'),
                        ('위험물취급자격자', 'Noun'),
                        ('지시', 'Noun'),
                        ('관련', 'Noun'),
                        ('의견', 'Noun'),
                        ('추진', 'Noun'),
                        ('특별시장', 'Noun'),
                        ('소통', 'Noun'),
                        ('방지', 'Noun'),
                        ('교육수료증', 'Noun'),
                        ("작업자", "Noun"),
                        ("예방", "Noun"),
                        ("대통령령", "Noun"),
                        ("취급자격자", "Noun"),
                        ("전문적", "Noun"),
                        ("지식", "Noun"),
                        ("능력", "Noun"),
                        ("관리", "Noun"),
                        ("화학물질", "Noun"),
                        ("법", "Noun"),
                        ("시설", "Noun"),
                        ("장비", "Noun"),
                        ("훈련", "Noun"),
                        ("유지보수", "Noun"),
                        ("이해도", "Noun"),
                        ("리더십", "Noun"),
                        ("생명", "Noun"),
                        ("분석", "Noun"),
                        ("대응책", "Noun"),
                        ("위험성", "Noun"),
                        ("수칙", "Noun"),
                        ("안전수칙", "Noun"),
                        ("프로그램", "Noun"),
                        ("운송자", "Noun"),
                        ("화재", "Noun"),
                        ("오염", "Noun"),
                        ("인명피해", "Noun"),
                        ("조작", "Noun"),
                        ("보관", "Noun"),
                        ("사용", "Noun"),
                        ('위험물', 'Noun'),
                        ('적재', 'Noun'),
                        ('운반방법', 'Noun'),
                        ('중요기준', 'Noun'),
                        ('세부기준', 'Noun'),
                        ('응급조치', 'Noun'),
                        ('영향', 'Noun'),
                        ('가능성', 'Noun'),
                        ('근거', 'Noun'),
                        ('시도지사', 'Noun'),
                        ('시행', 'Noun'),
                        ('국가기술자격법', 'Noun'),
                        ('운송과정', 'Noun'),
                        ('운송용기', 'Noun'),
                        ('지방', 'Noun'),
                        ('협조', 'Noun'),
                        ('정지명령', 'Noun'),
                        ('제거', 'Noun'),
                        ('긴급', 'Noun'),
                        ('방해', 'Noun'),
                        ('금지', 'Noun'),
                        ('제재', 'Noun'),
                        ('운영', 'Noun'),
                        ('직위', 'Noun'),
                        ('표시', 'Noun'),
                        ('협력', 'Noun'),
                        ('진압대책', 'Noun'),
                        ('정지', 'Noun'),
                        ('요구', 'Noun'),
                        ('자격증', 'Noun'),
                        ('국가기술자격증', 'Noun'),
                        ('전문지식', 'Noun'),
                        ('사람', 'Noun'),
                        ('도지사', 'Noun')  ]
for a_word, a_pos in my_new_words:
  tagger.add_dictionary(a_word, a_pos)

/usr/local/lib/python3.10/dist-packages/konlpy/tag/_okt.py:17: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [7]:
# 한글 불용어 사전:(28개)
my_stopwords = ['이란', '것입', '때',   '네',   '의',   '및',   '어떤',
                '그',   '때',   '데',   '중',   '따르', '두',   '이',
                '있',   '하',   '때문', '말하', '가지', '시키', '그러',
                '하나', '집',   '나',   '다른', '어떻', '들',   '말']

In [8]:
# 한글 분절 전처리 해주는 함수
def korean_processor(lines):
  res = []
  for a_line in lines:
    a_line = tagger.pos(a_line)
#    print(a_line)
    a_line_processed = ''
    for a_word, a_pos in a_line:
      if a_pos in ['Noun', 'Verb', 'Adjective'] and a_word not in my_stopwords:       # 불용어가 아니면서 품사가 명사, 동사, 형용사인 경우만 선택한다.
        a_line_processed += a_word + ' '
    if len(a_line_processed) > 10:
      res.append(a_line_processed)
  return res

In [9]:
# 한글 분절 전처리.
questions_processed = korean_processor(questions)

In [10]:
print(questions_processed)

['위험물안전관리법 적용 대상 무엇 ', '위험물 무엇 하나요 ', '저장소 취급소 차이 무엇 ', '위험물 저장 취급 제한 는가 ', '제조 소등 위험물 저장 취급 기준 적용 하나요 ', '위험물안전관리법 공공 안전 위해 목적 두는 이유 무엇 ', '제조 소등 위험물 저장 취급 기준 적용 하나요 ', '위험물안전관리법 목적 무엇 ', '제조소 등 위치 구조 설비 관 기술기준 누가 정하나 ', '위험물안전관리법 국가 책무 정 사항 무엇 ', '위험물안전관리법 적용 되지 않는 대상 것 있나요 ', '위험물 지정수량 무엇 의미 하나요 ', '위험물 저장소 취급소 기준 따라 구분 되나요 ', '위험물 저장 취급 제한 있는 경우 있나요 ', '위험물 저장소 취급소 안전 관리 기준 적용 하나요 ', '위험물안전관리법 지방자치단체 요구 하는 사항 무엇 ', '제조소 등 위치 구조 설비 대한 기술기준 정 하는 주체 누구 ', '위험물안전관리법 공공 안전 위해 두는 목적 무엇 ', '위험물 저장소 취급소 구분 하는 기준 무엇 ', '위험물안전관리법 국가 책무 정 사항 것 있나요 ', '위험물 저장 취급 시 출입 검사 할 수 있는 권한 누구 있나요 ', '주행 중인 위험물 운반 차량 정지 시 켜 국가기술자격증 교육수료증 요구 할 수 있는 권한 누구 있나요 ', '소방본부장 소방서장 출입 검사 명령 내릴 수 있는 대상 무엇 ', '위험물 저장소 발생 사고 원인 피해 조 사하 권한 누구 있나요 ', '소방공무원 이나 경찰공무원 주행 중인 위험물 운반 차량 정지 시 키 국가기술자격증 요구 하는 상황 국가기술자격증 제시 하지 않은 경우 공무원 요구 하는 것 무엇 ', '출입 검사 명령 받은 관계인 거부 경우 공무원 조치 취할 수 있나요 ', '위험물 저장 취급 시 출입 제한 하는 경우 개인 주거 대해 는 예외 적용 되나요 ', '소방본부장 소방서장 출입 검사 명령 내릴 수 있는 시간 언제 ', '출입 검사 명령 수행 하는 공무원 증표 지녀야 하나요 ', '위험물 저장소 발생 사고 조

In [11]:
# TF-IDF 모형 학습.
my_vectorizer = TfidfVectorizer(max_features = 300)
my_vectorizer.fit(questions_processed)

TfidfVectorizer(max_features=300)

In [12]:
# 사용된 feature들을 출력해 본다.
print(my_vectorizer.get_feature_names_out())

['가요' '가진' '간주' '감독' '강구' '개인' '거나' '거부' '거쳐야' '검사' '결정' '경우' '경찰공무원' '경하'
 '경험' '고하' '공공' '공무원' '과실' '과정' '과징금' '과태료' '관계인' '관련' '관리' '교육' '교육비'
 '교육수료증' '구분' '구성' '구조' '국가' '국가기술자격증' '군부대' '권고' '권한' '규정' '근거' '금액' '기간'
 '기관' '기술기준' '기재' '기준' '긴급' '긴밀' '납부' '내릴' '내용' '누가' '누구' '누설' '는가' '다수'
 '대리인' '대리자' '대상' '대상자' '대응' '대체' '대표자' '대한' '대해' '대행' '도록' '도지사' '되나요'
 '되는' '되면' '되어' '되지' '두는' '따라' '따르지' '따른' '라야' '려면' '루고' '명령' '명시' '목적'
 '무엇' '무허가장소' '문제' '물탱크' '받게' '받기' '받나요' '받는' '받아' '받으면' '받은' '받을' '받지'
 '발견' '발생' '방법' '방안' '방지' '벌금' '벌어질까' '범위' '법규' '법령' '법률' '법인' '법적' '변경'
 '변경공사' '변경허가' '보고' '보장' '보통' '부과' '부하' '분석' '비밀' '사고' '사고조사위원회' '사람' '사망'
 '사실' '사용' '사용정지' '사용정지명령' '사하' '사항' '선임' '설립' '설비' '설치' '설치자' '소등'
 '소방본부장' '소방서장' '소방청장' '수량' '수수료' '수행' '시간' '시설' '신고' '실시' '아야' '안전'
 '안전관리자' '안전교육' '안전사고' '안전조치' '않고' '않는' '않아' '않으면' '않은' '어디' '어떻게' '어질'
 '언제' '업무' '역할' '예외' '완공검사' '요건' '요구' '용도' '운반' '운반용기' '운송' '운송용기' '운송자'
 '운송책임자' '원인' '위반' '위치' '위탁' '위해' '위험' '위험물' '위험물안전관리법' '유지'

In [13]:
# 등록된 질문들을 TF-IDF 벡터로 변환.                                                                       (X는 여러개의 벡터들을 행으로하는 행렬)
X = my_vectorizer.transform(questions_processed).toarray()

In [14]:
# TF-IDF 벡터 한개 출력.
#print(X[0])
X.shape

(203, 300)

In [15]:
# 유저 입력.
user_input = '위험물 사고 발생 시 조사를 위해 구성되는 사고조사위원회의 구성은 어떻게 이루어지나요?'

In [16]:
# 유저 입력 분절 전처리.
user_input_processed = korean_processor([user_input])
print(user_input_processed)

['위험물 사고 발생 시 조사 위해 구성 되는 사고조사위원회 구성 어떻게 이루 지나 ']


In [17]:
# 유저 입력을 TF-IDF 벡터 1개로 변환.
user_x = my_vectorizer.transform(user_input_processed).toarray()[0]
print(user_x)

[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.61389726
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.27211908 0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.2168948
 0.         0.         0.         0.     

In [18]:
# 유저 입력과 등록된 질문사이의 코사인 유사도 계산.
css = []
for i in range(len(X)):
  cs = X[i].dot(user_x)          # 코사인 유사도.
  css.append(cs)

In [25]:
# 유저 입력과 등록된 질문사이의 코사인 유사도 계산.
css = []
for i in range(len(X)):
    cs = round(X[i].dot(user_x), 2)
    css.append(cs)

# 가장 높은 코사인 유사도를 보이는 경우 검출.
max_cos_similarities = sorted(css, reverse=True)[:5]  # 상위 5개의 코사인 유사도 가져오기
print(max_cos_similarities)


[1.0, 0.67, 0.42, 0.28, 0.21]


In [19]:
# 가장 높은 코사인 유사도를 보이는 경우 검출.
i_max = np.array(css).argmax()

In [22]:
max_cosine_similarity = max(css)
print("가장 높은 코사인 유사도:", max_cosine_similarity)

가장 높은 코사인 유사도: 1.0


In [20]:
# 유사도가 가장 높은 질문 출력.
print(end='인식된 질문 => ')
print(questions[i_max])

# 해당 answer 출력.
print(end='인식된 질문에 대한 답변 => ')
print(answers[i_max])

인식된 질문 => 위험물 사고 발생 시 조사를 위해 구성되는 사고조사위원회의 구성은 어떻게 이루어지나요?
인식된 질문에 대한 답변 => 위험물 사고 발생 시 조사를 위해 구성되는 사고조사위원회는 소방청장 등이 전문지식이 있는 사람으로 구성됩니다. (위험물안전관리법 제22조의2)


In [21]:
# 피클로 모델 저장
with open('chatbot_model.pkl', 'wb') as f:
    pickle.dump(my_vectorizer, f)
    pickle.dump(questions_processed, f)
    pickle.dump(answers, f)

[1.0, 0.67, 0.42, 0.28, 0.21]
